##### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2022 Semester 1

## Assignment 2: Sentiment Classification of Tweets

This is a sample code to assist you with vectorising the 'Train' dataset for your assignment 2.

First we read the CSV datafiles (Train and Test).

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

train_data = pd.read_csv("Train.csv", sep=',')
train_data.drop(train_data.filter(regex="Unname"),axis = 1,inplace=True)
test_data = pd.read_csv("Test.csv", sep=',')
X,y = train_data['text'],train_data['sentiment']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.28,random_state=42)

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
BoW_vectorizer = CountVectorizer(analyzer = 'word', ngram_range=(2,2))

#Build the feature set (vocabulary) and vectorise the Tarin dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_BoW = BoW_vectorizer.transform(X_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,2))

#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_raw)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw)

In [113]:
from sklearn.feature_selection import SelectKBest, chi2
x_new_chi = SelectKBest(chi2,k=20).fit_transform(X_train_BoW,y_train)
x_test_chi = SelectKBest(chi2,k=20).fit_transform(X_test_BoW,y_test)
x_test_chi.shape

(6105, 30000)

In [146]:
from sklearn.feature_selection import SelectKBest, f_classif
x_new_chi = SelectKBest(f_classif,k=20).fit_transform(X_train_BoW,y_train)
x_test_chi = SelectKBest(f_classif,k=20).fit_transform(X_test_BoW,y_test)

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
new_train_f = SelectKBest(mutual_info_classif,k=20).fit_transform(X_train_BoW,y_train)
new_test_f = SelectKBest(mutual_info_classif,k=20).transform(X_test)

In [42]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold= 0.0001)
x_var_train = selector.fit_transform(X_train_BoW)
x_var_test = selector.fit_transform(X_test_BoW)
x_var_train.shape

(15697, 25544)

Baseline-Model

In [43]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy= 'most_frequent')
model = clf.fit(x_var_train,y_train)
model.score(x_var_train,y_train)

0.5799834363254125

In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
lr = LogisticRegression(solver= 'sag', multi_class= 'multinomial', C = 0.7, max_iter = 200).fit(x_new_chi, y_train)
y_train_predicted = lr.predict(x_new_chi)
print("training accuracy ", np.mean(y_train_predicted== y_train))
lr_acc = np.mean(cross_val_score(lr,x_test_chi,y_test,cv = 5))
print('validation accuracy: ' ,lr_acc)
# y_train_predicted = lr.predict(x_test_chi)
# accuracy_score(lr.predict(x_test_chi),y_test)


training accurac:  0.8972415111167739
validation accuracy:  0.6024570024570025


In [132]:
from sklearn.svm import SVC
clf = SVC(kernel="rbf", C=10).fit(x_new_chi, y_train)
y_train_predicted = clf.predict(x_new_chi)
print("training accuracy ", np.mean(y_train_predicted == y_train))
clf_acc = np.mean(cross_val_score(clf,x_test_chi,y_test,cv = 5))
print('validation accuracy: ' ,clf_acc)

training accuracy  0.9745811301522584
validation accuracy:  0.6022932022932024


In [130]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear','rbf'),'C':[1,10]}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters)
clf.fit(x_new_chi, y_train)



GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [131]:
clf.best_params_

{'C': 10, 'kernel': 'rbf'}

In [135]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
tree = clf.fit(x_new_chi, y_train)
y_train_predicted = tree.predict(x_new_chi)
print("training accuracy ", np.mean(y_train_predicted== y_train))
tree_acc = np.mean(cross_val_score(tree,x_test_chi,y_test,cv = 5))
print('validation accuracy: ' ,tree_acc)

training accuracy  0.9798050582913933
validation accuracy:  0.6065520065520065


In [137]:
grid_pram = {
    "n_estimators": [90,100,115,130],
    'criterion':['gini','entropy'],
    'max_depth':range(2,20,1),
    'min_samples_leaf': range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features':['auto','log2']    

}
grid_search = GridSearchCV(clf, grid_pram,cv =5, n_jobs=-1, verbose=3 )
grid_search.fit(x_new_chi, y_train)


Fitting 5 folds for each of 20736 candidates, totalling 103680 fits
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90;, score=0.580 total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90;, score=0.580 total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90;, score=0.580 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90;, score=0.580 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90;, score=0.580 total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.580 total time=   0.5s
[CV 1/5] END criterion=gi

KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
estimator = []
